In [1]:
!pip install --quiet -U gensim

     |████████████████████████████████| 24.1 MB 1.1 MB/s 


In [2]:
!pip install --quiet transformers

     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.9 MB/s 
     |████████████████████████████████| 120 kB 65.6 MB/s 


In [3]:
!pip3 install --quiet keras

In [4]:
!pip3 install --quiet keras

In [5]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 7.5 MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd drive/MyDrive/AML/HW4/classifier

/content/drive/MyDrive/AML/HW4/classifier


In [8]:
!ls

 BERT-classification-on-text-augmented.ipynb
 BERT-classification-on-text.ipynb
 BERT-pretrained-features.ipynb
 classification_test_results.tsv
 dreaddit
 dreaddit-test.csv
 dreaddit-train.csv
 dreddit.ipynb
 GoogleNews-vectors-negative300.bin
 GoogleNews-vectors-negative300.bin.gz
 logs
 pre-trained-BERT-features-2.ipynb
 results
 tBERT_LDA_augmented_dreddit.ipynb
 tBERT_with_LDA_for_dreddit.ipynb
 test_data.tsv
 tfi-df-embeddings.ipynb
 train_data_for_students.tsv
 trump_tweets_classifiers.ipynb
'w2v_domain_trained_ GSDMM.ipynb'
 w2v-domain-trained.ipynb
 w2v_domain_trained_lda_augmented_self_training.ipynb
 w2v-domain-trained-lda.ipynb
 w2v_domain_trained_LDA_multilabled.ipynb
 w2v-google-news-pretrained.ipynb


In [9]:
# Data processing
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer

import transformers
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

# Visualization
from matplotlib import pyplot as plt

import nltk
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder

# Text processing
import re
import nltk
import gensim.downloader as api

# Various
from datetime import datetime
#Google Drive
from google.colab import drive

import re
import tqdm

import gensim, logging

import nlpaug.augmenter.word as naw

import gc
from torch.cuda import empty_cache

In [10]:
def set_seed(seed: int):
    # numpy
    np.random.seed(seed)

    # torch
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
         
set_seed(42)

In [11]:
def concat_two(original, new_x, features_from_original):
  new_df = pd.concat([new_x, original[features_from_original]], axis=1)
  return new_df

In [12]:
def get_domain(subreddit):
  if subreddit=='domesticviolence' or subreddit=='survivorsofabuse':
    return 'abuse';
  if subreddit=='anxiety' or subreddit=='stress':
    return 'anxiety';
  if subreddit=='almosthomeless' or subreddit=='assistance' or subreddit=='food_pantry' or subreddit=='homeless':
    return 'financial';
  if subreddit=='ptsd':
    return 'ptsd';
  else:
    return 'social';

In [13]:
df=pd.read_csv('dreaddit/dreaddit-train.csv')
df.shape

(2838, 116)

In [14]:
TRESHOLD = 0.8

In [15]:
df = df[df['confidence'] >= TRESHOLD]

In [16]:
df_test = pd.read_csv('dreaddit/dreaddit-test.csv')
df_test.shape

(715, 116)

In [17]:
df = df.reset_index()
df_test = df_test.reset_index()

In [18]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
# Reference: Text Classification with NLP: Tf-Idf vs Word2Vec vs BERT from Medium

# Use stopwords list from nltk
lst_stopwords = nltk.corpus.stopwords.words("english")

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    # Clean (convert to lowercase and remove punctuations and characters and then strip)
    # The function is not optimized for speed but split into various steps for pedagogical purpose
    text = str(text).lower()
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize (convert from string to list)
    lst_text = text.split()
    # remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    # Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    # Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    # back to string from list
    text = " ".join(lst_text)
    return text


In [20]:
df['text_clean'] = df["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=lst_stopwords))
df['domain'] = df['subreddit'].map(lambda x: get_domain(x))
X_train = df['text_clean']
y_train = df['label']
y_train = y_train.to_list()
df.head(2)

,index,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,text_clean,domain
0,0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,...,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742,said felt way suggeted go rest trigger ahead y...,ptsd
1,1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,...,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857,hey rassistance sure right place post goes im ...,financial


In [21]:
df_test['text_clean'] = df["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=lst_stopwords))
df_test['domain'] = df_test['subreddit'].map(lambda x: get_domain(x))
X_test = df_test['text_clean']
y_test = df_test['label']
y_test = y_test.to_list()
df_test.head(2)

,index,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,text_clean,domain
0,0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not.“ ME: I have...",0,0.8,1.514981e+09,22,...,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000,said felt way suggeted go rest trigger ahead y...,social
1,1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1.493348e+09,5,...,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909,hey rassistance sure right place post goes im ...,anxiety


In [22]:
ll = df[df.columns[1:]].corr().abs()['label'].sort_values(ascending = False)[:16].gt(0.389999).drop('label').where(lambda x : x == True).dropna()

In [23]:
top_correlations = list(ll.index)
top_correlations

['lex_liwc_Tone',
 'lex_liwc_Clout',
 'lex_liwc_negemo',
 'lex_liwc_i',
 'sentiment',
 'lex_dal_min_pleasantness']

In [24]:
train_for_vocabulary = df[df['confidence'] >= TRESHOLD]

In [25]:
import itertools
aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
augmented_texts_positive = [aug.augment(s, n=4) for s in train_for_vocabulary[train_for_vocabulary['label']==1].text.tolist()]
augmented_texts_positive = list(itertools.chain.from_iterable(augmented_texts_positive))
y_train_p = np.ones(len(augmented_texts_positive), dtype= np.int8).tolist()
augmented_text_negative = [aug.augment(s, n=5) for s in train_for_vocabulary[train_for_vocabulary['label']==0].text.tolist()]
augmented_text_negative = list(itertools.chain.from_iterable(augmented_text_negative))
y_train_n = np.zeros(len(augmented_text_negative), dtype= np.int8).tolist()
for_w2v_training = []
for_w2v_training.extend(augmented_texts_positive)
for_w2v_training.extend(augmented_text_negative)
for_w2v_training = [s for s in for_w2v_training]
y_train_augmented = []
y_train_augmented.extend(y_train_p)
y_train_augmented.extend(y_train_n)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [26]:
train_tuples = list(zip(for_w2v_training, y_train_augmented))
new_train_data = pd.DataFrame(train_tuples, columns =['text', 'label'])
new_train_data = new_train_data.sample(frac=1).reset_index(drop=True)
new_train_data.shape

(8257, 2)

In [27]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [28]:
class BERT_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        # print(item["labels"].shape)
        # 
        return item

    def __len__(self):
        return len(self.labels)

In [29]:
class BertTrainData:
  def __init__(self, X_train, X_test, y_train, y_test):
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

    self.train_texts = X_train
    self.valid_texts = X_val
    self.train_labels = y_train
    self.valid_labels = y_val
    self.test_text = X_test
    self.test_labels = y_test

In [30]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=20,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    load_best_model_at_end=True,    
    logging_steps=250,              
    evaluation_strategy="steps",
);

In [31]:
def get_prediction(text, max_length, targets, tokenizer, model):
    # tokenize the input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # get output of the fine-tuned model for this tokenized input
    outputs = model(**inputs)
    # convert model outputs to probabilities by performing a softmax
    probs = outputs[0].softmax(1)
    # use arg max to find the index where probabilty is the maximum, this index is our predicted label for the input text
    return targets[probs.argmax()]

def tain_bert_model(model_name, targets, max_length, train_data, do_lower_case):

    tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=do_lower_case)
    
    train_encodings = tokenizer(train_data.train_texts, truncation=True, padding=True, max_length=max_length)
    valid_encodings = tokenizer(train_data.valid_texts, truncation=True, padding=True, max_length=max_length)


    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(targets)).to("cuda")

    trainer = Trainer(
        model=model,                        
        args=training_args,                  
        train_dataset=BERT_Dataset(train_encodings, train_data.train_labels),        
        eval_dataset=BERT_Dataset(valid_encodings, train_data.valid_labels),          
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return model, tokenizer, trainer

In [32]:
test_size = 0.1
max_length = 512
targets = ['NoStressed','Stressed']

In [33]:
X_train = new_train_data['text']
y_train = new_train_data['label']
# X_train = df['text']
# y_train = np.ones(df['label'].shape[0], dtype= np.int8)
X_test = df_test['text']
y_test = df_test['label']

In [34]:
pretrained_model_name = "bert-base-uncased"

train_data = BertTrainData(X_train.tolist(), X_test.tolist(), y_train.tolist(), y_test.tolist())

model, tokenizer, trainer = tain_bert_model(pretrained_model_name, targets, max_length, train_data, False)

evaluation_result = trainer.evaluate()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy
250,0.449700,0.272738,0.902663
500,0.188900,0.119222,0.966102
750,0.084800,0.096708,0.977240
1000,0.020200,0.043131,0.992252


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-1000 (score: 0.0431305393576622).
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20


In [35]:
print(evaluation_result)

{'eval_loss': 0.0431305393576622, 'eval_accuracy': 0.9922518159806295, 'eval_runtime': 17.5363, 'eval_samples_per_second': 117.756, 'eval_steps_per_second': 5.931, 'epoch': 3.0}


In [36]:
predicted_labels_bert_cased = []
for instance in train_data.test_text:
  predicted_label = get_prediction(instance, max_length, targets, tokenizer, model)
  predicted_labels_bert_cased.append(predicted_label)
true_labels = [targets[y] for y in train_data.test_labels]
accuracy_bert_cased = accuracy_score(true_labels, predicted_labels_bert_cased)

print(classification_report(true_labels, predicted_labels_bert_cased))

              precision    recall  f1-score   support

  NoStressed       0.83      0.74      0.78       346
    Stressed       0.78      0.86      0.82       369

    accuracy                           0.80       715
   macro avg       0.80      0.80      0.80       715
weighted avg       0.80      0.80      0.80       715



In [37]:
model = None
gc.collect()
empty_cache()

In [38]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_labels, predicted_labels_bert_cased)

array([[257,  89],
       [ 53, 316]])

In [39]:
# 
pretrained_model_name = "mental/mental-bert-base-uncased"

model_, tokenizer_, trainer_ = tain_bert_model(pretrained_model_name, targets, max_length, train_data, False)

evaluation_result_ = trainer_.evaluate()

print(evaluation_result_)

https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7ox1e54q


storing https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/994b4764f990924cc4bf4fb7bb7af19f870d0fe2b491edcf1cad8a6c45e9e2b6.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/994b4764f990924cc4bf4fb7bb7af19f870d0fe2b491edcf1cad8a6c45e9e2b6.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp24acr7_c


storing https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/5821744706f4feb104635d11d3df0148f9b5e9cefc37106eddd374e5e0ce1d27.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/5821744706f4feb104635d11d3df0148f9b5e9cefc37106eddd374e5e0ce1d27.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp4vdyyjq5


storing https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/a79e898b922431c2669cf6bd2dcade813bc208bc3f1af50e92e8894ccd50bbcd.59407384618422b5f582b6046df91db98a0f921d6c959dc7b1f50000ffea1032
creating metadata file for /root/.cache/huggingface/transformers/a79e898b922431c2669cf6bd2dcade813bc208bc3f1af50e92e8894ccd50bbcd.59407384618422b5f582b6046df91db98a0f921d6c959dc7b1f50000ffea1032
loading file https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/994b4764f990924cc4bf4fb7bb7af19f870d0fe2b491edcf1cad8a6c45e9e2b6.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/special_tokens_map.json from cache at /root/.cache/hugg

storing https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ad6275274e5e6b90d918940d141be81592e14a9db4f8cef900b59ac683ffb0cc.c3d226abd353aca9215597361dcd31f338854c9273effd5d9120062d49a2b932
creating metadata file for /root/.cache/huggingface/transformers/ad6275274e5e6b90d918940d141be81592e14a9db4f8cef900b59ac683ffb0cc.c3d226abd353aca9215597361dcd31f338854c9273effd5d9120062d49a2b932
loading configuration file https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ad6275274e5e6b90d918940d141be81592e14a9db4f8cef900b59ac683ffb0cc.c3d226abd353aca9215597361dcd31f338854c9273effd5d9120062d49a2b932
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hid

storing https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/5ca41d7cb20147525c3f96b967f06302d4b46bb86f11cca6393fe0229b47b701.8c88c9e07620cc8180f76050ae6c0e3d5beeec39e16368f47fa7659a3f90e512
creating metadata file for /root/.cache/huggingface/transformers/5ca41d7cb20147525c3f96b967f06302d4b46bb86f11cca6393fe0229b47b701.8c88c9e07620cc8180f76050ae6c0e3d5beeec39e16368f47fa7659a3f90e512
loading weights file https://huggingface.co/mental/mental-bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/5ca41d7cb20147525c3f96b967f06302d4b46bb86f11cca6393fe0229b47b701.8c88c9e07620cc8180f76050ae6c0e3d5beeec39e16368f47fa7659a3f90e512
Some weights of the model checkpoint at mental/mental-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

Step,Training Loss,Validation Loss,Accuracy
250,0.383500,0.189836,0.928814
500,0.143600,0.091910,0.973850
750,0.084500,0.081712,0.983051
1000,0.015600,0.065098,0.989346


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-1000 (score: 0.0650981143116951).
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 20


{'eval_loss': 0.0650981143116951, 'eval_accuracy': 0.9893462469733656, 'eval_runtime': 17.5337, 'eval_samples_per_second': 117.773, 'eval_steps_per_second': 5.931, 'epoch': 3.0}


In [40]:
predicted_labels_bert_cased_ = []
for instance in train_data.test_text:
  predicted_label = get_prediction(instance, max_length, targets, tokenizer_, model_)
  predicted_labels_bert_cased_.append(predicted_label)
true_labels = [targets[y] for y in train_data.test_labels]
accuracy_bert_cased = accuracy_score(true_labels, predicted_labels_bert_cased_)

print(classification_report(true_labels, predicted_labels_bert_cased_))

              precision    recall  f1-score   support

  NoStressed       0.88      0.72      0.79       346
    Stressed       0.78      0.91      0.84       369

    accuracy                           0.82       715
   macro avg       0.83      0.81      0.81       715
weighted avg       0.83      0.82      0.81       715



In [41]:
model_ = None
gc.collect()
empty_cache()

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_labels, predicted_labels_bert_cased_)

array([[249,  97],
       [ 34, 335]])